In [ ]:
! pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-h72btexj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-h72btexj
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [ ]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [ ]:
import whisper

model_name = "large"   # tiny | base | small | medium | large
model = whisper.load_model(model_name, DEVICE)
script = model.transcribe("test.wav")

100%|██████████████████████████████████████| 2.88G/2.88G [00:23<00:00, 130MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
script['text'][:1000]

" So, Erin, in your email you said you wanted to talk about the exam. Yeah, um, I've just never taken a class with so many different readings. I've managed to keep up with all the assignments, but I'm not sure how to... how to... How to review everything? Yeah. In other classes I've had, there's usually just one book to review, not three different books, plus all those other text excerpts and videos."

In [ ]:
! pip install git+https://github.com/m-bain/whisperX.git

  Cloning https://github.com/m-bain/whisperX.git to /tmp/pip-req-build-3izemen2
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperX.git /tmp/pip-req-build-3izemen2
  Resolved https://github.com/m-bain/whisperX.git to commit 58f00339af7dcc9705ef49d97a1f40764b7cf555
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
HF_TOKEN = "YOUR_API_KEY"

In [ ]:
from whisperx.diarize import DiarizationPipeline

diarization_pipeline = DiarizationPipeline(use_auth_token=HF_TOKEN)

In [ ]:
diarized = diarization_pipeline("/content/test.wav")

In [ ]:
diarized

,segment,label,speaker,start,end
0,[ 00:00:00.297 --> 00:00:04.168],A,SPEAKER_01,0.297114,4.168081
1,[ 00:00:04.881 --> 00:00:09.940],B,SPEAKER_00,4.881154,9.940577
2,[ 00:00:10.110 --> 00:00:12.555],C,SPEAKER_00,10.110357,12.555178
3,[ 00:00:12.945 --> 00:00:14.286],D,SPEAKER_00,12.945671,14.286927
4,[ 00:00:14.966 --> 00:00:15.560],E,SPEAKER_00,14.966044,15.560272
5,[ 00:00:16.205 --> 00:00:18.073],F,SPEAKER_02,16.205433,18.073005
6,[ 00:00:18.429 --> 00:00:19.057],G,SPEAKER_00,18.429542,19.057725
7,[ 00:00:19.363 --> 00:00:20.891],H,SPEAKER_00,19.363328,20.891341
8,[ 00:00:21.078 --> 00:00:27.427],I,SPEAKER_00,21.078098,27.427844


In [ ]:
from whisperx import load_align_model, align
from whisperx.diarize import assign_word_speakers

In [ ]:
model_a, metadata = load_align_model(language_code=script["language"], device=DEVICE)
script_aligned = align(script["segments"], model_a, metadata, "/content/test.wav", DEVICE)

# Align Speakers
result_segments, word_seg = list(assign_word_speakers(
    diarized, script_aligned
).values())
transcribed = []
for result_segment in result_segments:
    transcribed.append(
        {
            "start": result_segment["start"],
            "end": result_segment["end"],
            "text": result_segment["text"],
            "speaker": result_segment["speaker"],
        }
    )

In [ ]:
for start, end, text, speaker in [i.values() for i in transcribed]:
    print(start, end, speaker, text)

0.402 3.92 SPEAKER_01  So, Erin, in your email you said you wanted to talk about the exam.
4.94 9.38 SPEAKER_00  Yeah, um, I've just never taken a class with so many different readings.
10.221 15.158 SPEAKER_00  I've managed to keep up with all the assignments, but I'm not sure how to... how to...
16.224 17.8 SPEAKER_02  How to review everything?
18.48 18.921 SPEAKER_00  Yeah.
19.442 26.94 SPEAKER_00 In other classes I've had, there's usually just one book to review, not three different books, plus all those other text excerpts and videos.
